In [8]:
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

In [5]:
# Parâmetros de conexão com o PostgreSQL
db_host = 'localhost'
db_port = os.getenv('DB_EXTERNAL_PORT')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

In [6]:
try:
    # Testar conexão antes de prosseguir
    test_conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_database,
        user=db_user,
        password=db_password
    )
    print("Conexão com o PostgreSQL estabelecida com sucesso!")
    test_conn.close()
except Exception as e:
    print(f"Erro ao estabelecer conexão com o PostgreSQL: {e}")

Conexão com o PostgreSQL estabelecida com sucesso!


In [15]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}')

query = """-- Listar Tabelas
SELECT table_schema, table_name, 'table' AS type
FROM information_schema.tables
WHERE table_schema IN ('bronze', 'silver', 'gold')
  AND table_type = 'BASE TABLE'

UNION

-- Listar Views
SELECT table_schema, table_name, 'view' AS type
FROM information_schema.views
WHERE table_schema IN ('bronze', 'silver', 'gold')

UNION

-- Listar Views Materializadas
SELECT schemaname, matviewname AS table_name, 'materialized view' AS type
FROM pg_matviews
WHERE schemaname IN ('bronze', 'silver', 'gold')
ORDER BY table_schema, table_name;
"""
available_tables = pd.read_sql(query, con=engine)
available_tables

,table_schema,table_name,type
0,bronze,brt_data,table
1,gold,vw_brt_last_info,materialized view
